In [11]:
!pwd

/workspace/code/gym-continuousDoubleAuction/gym_continuousDoubleAuction


In [12]:
%run ../gym_continuousDoubleAuction/test/test_accounting.py

.............
----------------------------------------------------------------------
Ran 13 tests in 0.007s

OK


In [13]:
%run ../gym_continuousDoubleAuction/test/test_orderbook.py

............
----------------------------------------------------------------------
Ran 12 tests in 0.006s

OK


In [14]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_double_delete_order.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [15]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_volume_sync.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [16]:
%run ../gym_continuousDoubleAuction/test/test_orderbook_crossed_book.py

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [17]:
%run ../gym_continuousDoubleAuction/test/test_modify_order.py

......
----------------------------------------------------------------------
Ran 6 tests in 0.003s

OK


In [18]:
# %run ../gym_continuousDoubleAuction/test/test_nav_callback.py

In [19]:
%run ../gym_continuousDoubleAuction/test/test_new_action_space.py

........
----------------------------------------------------------------------
Ran 8 tests in 0.013s

OK


In [20]:
%run ../gym_continuousDoubleAuction/inspect_latest_episode.py

Latest file found: ../gym_continuousDoubleAuction/episode_data/f5887d99e77f47b6ab7bb55ef30bfada.pkl

Data successfully written to latest_episode_data.txt


In [21]:
%run ../gym_continuousDoubleAuction/inspect_latest_episode_json.py

Latest file found: ../gym_continuousDoubleAuction/episode_data/f5887d99e77f47b6ab7bb55ef30bfada.pkl

Data successfully written to latest_episode_data.json


In [22]:
# %run ../gym_continuousDoubleAuction/analyze_unused.py

In [23]:
from datetime import datetime

now = datetime.now()
formatted = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted)  # e.g., "2025-12-22 10:30:45"

2025-12-24 10:09:11
